In [ ]:
from IPython.display import display, Math, Latex

import pandas as pd
import numpy as np
import numpy_financial as npf
import yfinance as yf
import matplotlib.pyplot as plt
from datetime import datetime

In [ ]:
# Reads the Tickers.csv file, and outputs a dataframe of all tickers. 
tickers = pd.read_csv("Tickers.csv")

In [ ]:
# Creating an empty list, where the ticker names will be stored. 
tickers_lst = []

# A for loop to append all tickers from the DataFrame into a list
for i in range(len(tickers)):
    tickers_lst.append(tickers.iloc[i][0])

# One last ticker remaining in the column header, so also append that to the list 
tickers_lst.append(tickers.columns.values[0])

In [ ]:
# Create an empty list where valid tickers will be stored 
valid_tickers = []
# For loop to check if each ticker is valid according to instructions, if true then append to the list 
for each_ticker in tickers_lst:
    stock = yf.Ticker(each_ticker)
    avg_volume= stock.history(start="2021-07-02", end = "2021-10-21").Volume.mean()
    if avg_volume >= 10000 and stock.info['market'] == 'us_market':
        valid_tickers.append(each_ticker)

In [ ]:
# Variables for start dates and end dates to get the history of the valid tickers 
start_date='2018-11-20' 
end_date="2021-11-20"

In [ ]:
# Creates a dictionary with the tickers as keys and their closing prices as values
ticker_hist_dict = {}
for each_ticker in valid_tickers:
    ticker = yf.Ticker(each_ticker)
    close = ticker.history(start=start_date, end=end_date).Close
    ticker_hist_dict.update({each_ticker: close})

In [ ]:
# Creating an empty list that will store the tickers of our final portfolio 
portfolio = []

In [ ]:
# Adding the the stock with maximum standard deviation as our first stock in the portfolio
ticker_hist_df = pd.DataFrame(ticker_hist_dict)
std = pd.DataFrame(ticker_hist_df.pct_change().std())
std.columns = ["Standard deviations"]
max_std = max(ticker_hist_df.pct_change().std())
for index, row in std.iterrows():
    if max_std == row["Standard deviations"]:
        portfolio.append(index)

In [ ]:
# Creating a dictionary that will store the correlation between all the remaining stocks
# and the stock with the maximum standard deviation in the portfolio that was previously chosen
ticker = yf.Ticker(portfolio[0]).history(start=start_date, end =end_date).Close
correlation = {}
ticker_df = pd.DataFrame(ticker)
for each_ticker in valid_tickers: 
    if each_ticker != portfolio[0]:
        ticker_df[each_ticker] = yf.Ticker(each_ticker).history(start=start_date, end =end_date).Close      
        correlation.update({each_ticker:ticker_df.Close.corr(ticker_df[each_ticker])})
correlation

In [ ]:
# Choosing the remaining 9 stocks according to their correlation with the first stock
for i in range(9):
    portfolio.append(max(correlation, key=correlation.get))
    del(correlation[max(correlation, key=correlation.get)])

In [ ]:
# printing out our final portfolio 
portfolio

In [ ]:
# Defining function that returns a dataframe with tickers and their closing prices
def history(list_of_tickers, start_date, end_date):
    hist_dict = {} 
    for ticker in list_of_tickers:
        stock = yf.Ticker(ticker)
        hist = stock.history(start=start_date, end=end_date, interval="1mo").Close
        hist_dict.update({ticker: hist})

    return pd.DataFrame(hist_dict).dropna()

In [ ]:
# Defining function that caclulates number of shares based on list_of_pcts that contains the weighting of each stock in list_of_tickers
def num_shares_pct(list_of_tickers, hist_df, list_of_pcts):
    num_shares = []
    i = 0
    for ticker in list_of_tickers:
        stock_price = hist_df.loc[:, ticker][0]
        num_shares.append((100000*(list_of_pcts[i]/100))/(stock_price))
        i += 1
    return num_shares

In [ ]:
# The weighting of each stock in our portfolio
percentages = [35, 25, 5, 5, 5, 5, 5, 5, 5, 5]

In [ ]:
# extracting history of the stocks in our portolio
hist_df = history(portfolio, start_date, end_date)

In [ ]:
# The date we will buy our portfolio on
buy_date = "2021-11-26"

# Creating a list with the buying price of all the stocks in our portfolio
buy_price = []
for each_ticker in portfolio:
    closing_price = yf.Ticker(each_ticker).history(start=buy_date).Close.values[0]
    buy_price.append(closing_price)

In [ ]:
# The number of shares bought for each stock based on our start_date 
shares = num_shares_pct(portfolio, hist_df, percentages)
shares

In [ ]:
# Defining function that calculates the portfolio based on the num of shares of each ticker in the list_of_tickers 
def portfolio_calc(list_of_tickers, hist_df, num_shares):
    value = {}
    n = 0
    for ticker in list_of_tickers:
        value.update({ticker: hist_df[ticker]*num_shares[n]})
        n += 1

    each_stock_value = pd.DataFrame(value)
    return pd.DataFrame({"portfolio": each_stock_value.sum(axis=1)})

# Creating a dataframe with the portfolio value
portfolio_df = portfolio_calc(portfolio, hist_df, shares)

# Joining history of each ticker in portfolio with portfolio_df
hist_w_portfolio= pd.concat([hist_df, portfolio_df], join='inner', axis=1)

In [ ]:
hist_w_portfolio

In [ ]:
# Calculating Beta
MarketIndex_hist = history(["^GSPC"], start_date, end_date)
index_w_portfolio = pd.concat([portfolio_df,MarketIndex_hist], join='inner', axis=1)

percent_change = index_w_portfolio.pct_change()
percent_change.drop(index=percent_change.index[0], inplace=True)

MarketVar = percent_change["^GSPC"].var()
MarketVar
Beta = percent_change.cov()/MarketVar
Beta

In [ ]:
# Calculating sharpe ratio assuming the risk free rate of return is 0
sharpe_ratio = portfolio_df.pct_change().mean()/portfolio_df.pct_change().std()
sharpe_ratio

In [ ]:
# Defining function that caclulates number of shares to be bought for each stock based on the prices in list_of_prices
def num_shares_lst(list_of_tickers, list_of_prices, list_of_pcts):
    num_shares = []
    i = 0
    for ticker in list_of_tickers:
        num_shares.append((100000*(list_of_pcts[i]/100))/(list_of_prices[i]))
        i += 1
    return num_shares

In [ ]:
# Displaying the price we are going to buy the stocks at
buy_price  

In [ ]:
# Calculating the number of shares bought
buy_shares = num_shares_lst(portfolio, buy_price, percentages)
buy_shares

In [ ]:
# Creating dictionary with FinalPortfolio data
FinalPortfolio_dict = {"index": [1, 2, 3, 4, 5, 6, 7, 8, 9, 10],
                 "Ticker": portfolio,
                       "Price": buy_price,
                       "Shares": buy_shares,
                      }
# Creating Dataframe from FinalPortfolio_dict
FinalPortfolio = pd.DataFrame(FinalPortfolio_dict) 
FinalPortfolio.set_index("index", inplace=True)
FinalPortfolio['Value'] = FinalPortfolio['Price']*FinalPortfolio['Shares']
FinalPortfolio["Weight"] = percentages
print("The total value of the portfolio is: $"+ str(FinalPortfolio.Value.sum(axis=0)))

In [ ]:
print("The total weight of the portfolio is: "+ str(FinalPortfolio.Weight.sum(axis=0))+"%")

In [ ]:
FinalPortfolio

In [ ]:
# Creating dictionary with Stocks data
Stocks_dict = {"index": [1, 2, 3, 4, 5, 6, 7, 8, 9, 10],
               "Ticker": portfolio,
               "Shares": buy_shares,}  
# Creating Dataframe from Stocks_dict 
Stocks = pd.DataFrame(Stocks_dict)
Stocks.set_index("index", inplace=True)
Stocks

In [ ]:
# Export final dataframe to a CSV file 
Stocks.to_csv('Stocks_Group_9.csv', index=False)